In [ ]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score, classification_report
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
%%time
idee = pd.read_csv('train_identity.csv')
transaction = pd.read_csv('train_transaction.csv')

In [ ]:
merge = transaction.merge(idee, how='outer', on='TransactionID')
objects = merge.select_dtypes('object')
objects = objects.join(merge[['card1', 'card2', 'card3', 'card5', 'addr1', 'addr2']])
objects_columns = objects.columns.to_list()
